In [3]:
import pandas as pd
import tensorflow as tf 
from keras.datasets import cifar10
import tensorflow.keras as K
from keras import backend as Kb
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D,BatchNormalization,AveragePooling2D
from keras.utils.vis_utils import plot_model
from keras import utils as np_utils
from tensorflow.keras.utils import to_categorical

In [4]:
#Preprocessing the data
def preprocess_data(X,Y):
  X_p = K.applications.xception.preprocess_input(X)
  Y_p = K.utils.to_categorical(Y,10)
  return X_p, Y_p

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_rows, img_cols = 32, 32
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)

170498071/170498071 [==============================] - 77s 0us/step
x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 1)


In [6]:
# pooling removing the boders
if Kb.image_data_format() == 'channels_first':
  x_train = x_train.reshape(x_train.shape[0], 3, img_rows, img_cols)
  x_test = x_test.reshape(x_test.shape[0], 3, img_rows, img_cols)
  input_shape = (3, img_rows, img_cols)
else:
  x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
  x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
  input_shape = (img_rows, img_cols, 3)

In [7]:
#Preprocessing the data
x_train, y_train = preprocess_data(x_train, y_train)
x_test, y_test = preprocess_data(x_test, y_test)

In [8]:
# Resizing all the images
input_t = K.Input(shape=(32,32,3))
xception_model = K.applications.Xception(include_top=False,
                                    weights="imagenet",
                                    input_tensor=input_t)

83683744/83683744 [==============================] - 4s 0us/step


In [9]:
#Each layer 
for layer in xception_model.layers[:10]:
  layer.trainable=False

In [10]:
for i, layer in enumerate(xception_model.layers):
  print(i,layer.name,"-",layer.trainable)

0 input_1 - False
1 block1_conv1 - False
2 block1_conv1_bn - False
3 block1_conv1_act - False
4 block1_conv2 - False
5 block1_conv2_bn - False
6 block1_conv2_act - False
7 block2_sepconv1 - False
8 block2_sepconv1_bn - False
9 block2_sepconv2_act - False
10 block2_sepconv2 - True
11 block2_sepconv2_bn - True
12 conv2d - True
13 block2_pool - True
14 batch_normalization - True
15 add - True
16 block3_sepconv1_act - True
17 block3_sepconv1 - True
18 block3_sepconv1_bn - True
19 block3_sepconv2_act - True
20 block3_sepconv2 - True
21 block3_sepconv2_bn - True
22 conv2d_1 - True
23 block3_pool - True
24 batch_normalization_1 - True
25 add_1 - True
26 block4_sepconv1_act - True
27 block4_sepconv1 - True
28 block4_sepconv1_bn - True
29 block4_sepconv2_act - True
30 block4_sepconv2 - True
31 block4_sepconv2_bn - True
32 conv2d_2 - True
33 block4_pool - True
34 batch_normalization_2 - True
35 add_2 - True
36 block5_sepconv1_act - True
37 block5_sepconv1 - True
38 block5_sepconv1_bn - True
39 b

In [11]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (3,3), padding='same', input_shape = input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 32, kernel_size = (3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))

model.add(Conv2D(filters = 64, kernel_size = (3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))

model.add(Conv2D(filters = 128, kernel_size = (3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 128, kernel_size = (3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.4))

model.add(Flatten())

model.add(Dense(units = 10, activation = 'softmax'))

In [12]:
model.compile(loss='categorical_crossentropy',
                  optimizer=K.optimizers.RMSprop(lr=2e-5),
                  metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=64, epochs=20, verbose=1, validation_data=(x_test, y_test),shuffle=True)
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/20
782/782 [==============================] - 200s 252ms/step - loss: 1.7659 - accuracy: 0.4672 - val_loss: 1.2081 - val_accuracy: 0.5916
Epoch 2/20
782/782 [==============================] - 198s 253ms/step - loss: 1.1023 - accuracy: 0.6410 - val_loss: 1.5177 - val_accuracy: 0.6533
Epoch 3/20
782/782 [==============================] - 189s 242ms/step - loss: 0.8686 - accuracy: 0.7083 - val_loss: 0.7950 - val_accuracy: 0.7336
Epoch 4/20
782/782 [==============================] - 314s 401ms/step - loss: 0.7374 - accuracy: 0.7499 - val_loss: 0.8324 - val_accuracy: 0.7352
Epoch 5/20
782/782 [==============================] - 209s 267ms/step - loss: 0.6495 - accuracy: 0.7797 - val_loss: 0.6467 - val_accuracy: 0.7840
Epoch 6/20
782/782 [==============================] - 204s 261ms/step - loss: 0.5915 - accuracy: 0.7969 - val_loss: 0.6302 - val_accuracy: 0.7915
Epoch 7/20
782/782 [==============================] - 431s 550ms/step - loss: 0.5488 - accuracy: 0.8117 - val_loss: 0.6054 -